In [1]:
import os
import gym #open ai gym lib that build environments
from stable_baselines3 import PPO #ppo=algorith famma o5rin
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3.common.evaluation import evaluate_policy

# Load Environment

In [2]:
environment_name = "CartPole-v0"

In [ ]:
env = gym.make(environment_name)

In [ ]:
env = gym.make(environment_name)
episodes = 5 # play the game fully 5 times
for episode in range(1, episodes+1):
    state = env.reset() #reset game
    done = False
    score = 0 
    
    while not done:
        env.render()
        action = env.action_space.sample()
        n_state, reward, done, info = env.step(action) #.step faire l'action
        score+=reward
    print('Episode:{} Score:{}'.format(episode, score))
env.close()

In [5]:
env.close()

# Understanding The Environment


In [6]:
env.action_space#how much action are available

Discrete(2)

In [7]:
env.action_space.sample() # random actions done, honi 0 et 1 5ater famma ken 2 actions

0

In [8]:
env.observation_space # check documentation of each env to understand values

Box([-4.8000002e+00 -3.4028235e+38 -4.1887903e-01 -3.4028235e+38], [4.8000002e+00 3.4028235e+38 4.1887903e-01 3.4028235e+38], (4,), float32)

In [9]:
env.observation_space.sample() #obszrvation of our actions

array([ 7.0232207e-01, -1.6415352e+38, -3.2849231e-01,  1.5377391e+38],
      dtype=float32)

In [11]:
env.step(1) # get observations / reward 1 if success 0 if failure/ if episode is done

C:\ProgramData\Anacondaa3\lib\site-packages\gym\logger.py:34: UserWarning: WARN: You are calling 'step()' even though this environment has already returned done = True. You should always call 'reset()' once you receive 'done = True' -- any further steps are undefined behavior.
  warnings.warn(colorize("%s: %s" % ("WARN", msg % args), "yellow"))


(array([ 0.05261654, -0.12792282, -0.21031227, -0.30009271]), 0.0, True, {})

# Train

In [11]:
paths = {
    'LOG_PATH': os.path.join('Training', 'Logs'),
    'SAVE': os.path.join('Training', 'Save Models','PPO cardpole'),
    'log': os.path.join(paths['LOG_PATH'], 'PPO_1'),
    
 }


In [22]:
env = gym.make(environment_name)
env = DummyVecEnv([lambda: env]) #wrapping the env 
model = PPO('MlpPolicy', env, verbose = 1,tensorboard_log=paths['LOG_PATH']) #mlp policy is the rule on how to opetate the env

Using cuda device


In [23]:
model.learn(total_timesteps=20000)

Logging to Training\Logs\PPO_2
-----------------------------
| time/              |      |
|    fps             | 560  |
|    iterations      | 1    |
|    time_elapsed    | 3    |
|    total_timesteps | 2048 |
-----------------------------
-----------------------------------------
| time/                   |             |
|    fps                  | 455         |
|    iterations           | 2           |
|    time_elapsed         | 8           |
|    total_timesteps      | 4096        |
| train/                  |             |
|    approx_kl            | 0.008990911 |
|    clip_fraction        | 0.101       |
|    clip_range           | 0.2         |
|    entropy_loss         | -0.686      |
|    explained_variance   | -0.0023     |
|    learning_rate        | 0.0003      |
|    loss                 | 7.66        |
|    n_updates            | 10          |
|    policy_gradient_loss | -0.015      |
|    value_loss           | 48.7        |
-----------------------------------------
---

# Save and Load

In [24]:
model.save(paths['SAVE'])

In [7]:
model=PPO.load(paths['SAVE'])

# test and evaluate

In [20]:
#evalute

In [9]:
evaluate_policy(model,env,n_eval_episodes=10,render=False)

C:\ProgramData\Anacondaa3\lib\site-packages\stable_baselines3\common\evaluation.py:65: UserWarning: Evaluation environment is not wrapped with a ``Monitor`` wrapper. This may result in reporting modified episode lengths and rewards, if other wrappers happen to modify these. Consider wrapping environment first with ``Monitor`` wrapper.
  warnings.warn(


(200.0, 0.0)

In [10]:
#test

In [8]:
env = gym.make(environment_name)
episodes = 5 # play the game fully 5 times
for episode in range(1, episodes+1):
    obs = env.reset() #reset game
    done = False
    score = 0 
    
    while not done:
        env.render()
        action,_ = model.predict(obs)
        obs, reward, done, info = env.step(action) #.step faire l'action
        score+=reward
    print('Episode:{} Score:{}'.format(episode, score))
env.close()

Episode:1 Score:200.0
Episode:2 Score:200.0
Episode:3 Score:200.0
Episode:4 Score:200.0
Episode:5 Score:200.0


# get Tensorboards

In [12]:
log=paths['log']

In [13]:
log

'Training\\Logs\\PPO_1'

In [14]:
!tensorboard --logdir=(log)

Traceback (most recent call last):
  File "C:\ProgramData\Anacondaa3\lib\site-packages\pkg_resources\__init__.py", line 568, in _build_master
    ws.require(__requires__)
  File "C:\ProgramData\Anacondaa3\lib\site-packages\pkg_resources\__init__.py", line 886, in require
    needed = self.resolve(parse_requirements(requirements))
  File "C:\ProgramData\Anacondaa3\lib\site-packages\pkg_resources\__init__.py", line 777, in resolve
    raise VersionConflict(dist, req).with_context(dependent_req)
pkg_resources.ContextualVersionConflict: (keras 2.6.0 (c:\programdata\anacondaa3\lib\site-packages), Requirement.parse('keras<2.9,>=2.8.0rc0'), {'tensorflow'})

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\ProgramData\Anacondaa3\Scripts\tensorboard-script.py", line 33, in <module>
    sys.exit(load_entry_point('tensorflow==2.8.0rc0', 'console_scripts', 'tensorboard')())
  File "C:\ProgramData\Anacondaa3\Scripts\tensorboard-scrip

#  Callbacks ( stop the learning when reaching a good threshhold before going unstable

In [17]:
from stable_baselines3.common.callbacks import EvalCallback,StopTrainingOnRewardThreshold

In [22]:
stop_callback=StopTrainingOnRewardThreshold(reward_threshold=200,verbose=1)
eval_callback=EvalCallback(env,
                          callback_on_new_best=stop_callback,
                          eval_freq=10000,
                          best_model_save_path=paths['SAVE'],
                          verbose=1)

In [24]:
model=PPO('MlpPolicy',env,verbose=1,tensorboard_log=paths['LOG_PATH'])

Using cuda device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


In [25]:
model.learn(total_timesteps=20000,callback=eval_callback)

Logging to Training\Logs\PPO_3
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 24.2     |
|    ep_rew_mean     | 24.2     |
| time/              |          |
|    fps             | 216      |
|    iterations      | 1        |
|    time_elapsed    | 9        |
|    total_timesteps | 2048     |
---------------------------------
------------------------------------------
| rollout/                |              |
|    ep_len_mean          | 28.2         |
|    ep_rew_mean          | 28.2         |
| time/                   |              |
|    fps                  | 275          |
|    iterations           | 2            |
|    time_elapsed         | 14           |
|    total_timesteps      | 4096         |
| train/                  |              |
|    approx_kl            | 0.0083559025 |
|    clip_fraction        | 0.101        |
|    clip_range           | 0.2          |
|    entropy_loss         | -0.686       |
|    explained_variance   |

C:\ProgramData\Anacondaa3\lib\site-packages\stable_baselines3\common\evaluation.py:65: UserWarning: Evaluation environment is not wrapped with a ``Monitor`` wrapper. This may result in reporting modified episode lengths and rewards, if other wrappers happen to modify these. Consider wrapping environment first with ``Monitor`` wrapper.
  warnings.warn(


Eval num_timesteps=10000, episode_reward=199.20 +/- 1.60
Episode length: 199.20 +/- 1.60
------------------------------------------
| eval/                   |              |
|    mean_ep_length       | 199          |
|    mean_reward          | 199          |
| time/                   |              |
|    total_timesteps      | 10000        |
| train/                  |              |
|    approx_kl            | 0.0073685823 |
|    clip_fraction        | 0.0815       |
|    clip_range           | 0.2          |
|    entropy_loss         | -0.61        |
|    explained_variance   | 0.277        |
|    learning_rate        | 0.0003       |
|    loss                 | 26.5         |
|    n_updates            | 40           |
|    policy_gradient_loss | -0.0199      |
|    value_loss           | 69.7         |
------------------------------------------
New best mean reward!
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 60.1     |
|    ep_rew_m

# Change policy ( try new NN)

In [28]:
#new_arch=[dict(pi=[128, 128, 128, 128], vf=[128, 128, 128, 128])]
new_arch=[128,128]

In [32]:
model=PPO('MlpPolicy',env,verbose=1,tensorboard_log=paths['LOG_PATH'],policy_kwargs={'net_arch':new_arch})

Using cuda device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


In [33]:
model.learn(total_timesteps=20000,callback=eval_callback)

Logging to Training\Logs\PPO_4
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 21.5     |
|    ep_rew_mean     | 21.5     |
| time/              |          |
|    fps             | 620      |
|    iterations      | 1        |
|    time_elapsed    | 3        |
|    total_timesteps | 2048     |
---------------------------------
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 26.3        |
|    ep_rew_mean          | 26.3        |
| time/                   |             |
|    fps                  | 512         |
|    iterations           | 2           |
|    time_elapsed         | 7           |
|    total_timesteps      | 4096        |
| train/                  |             |
|    approx_kl            | 0.008591624 |
|    clip_fraction        | 0.0939      |
|    clip_range           | 0.2         |
|    entropy_loss         | -0.687      |
|    explained_variance   | -0.00698    |

---------------------------------
| rollout/           |          |
|    ep_len_mean     | 99.1     |
|    ep_rew_mean     | 99.1     |
| time/              |          |
|    fps             | 424      |
|    iterations      | 10       |
|    time_elapsed    | 48       |
|    total_timesteps | 20480    |
---------------------------------


# Alternate algot

In [34]:
from stable_baselines3 import DQN

In [35]:
model=DQN('MlpPolicy',env,verbose=1,tensorboard_log=paths['LOG_PATH'],policy_kwargs={'net_arch':new_arch})

Using cuda device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


In [38]:
model.learn(total_timesteps=20000,callback=eval_callback)

Logging to Training\Logs\DQN_3
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 35.2     |
|    ep_rew_mean      | 35.2     |
|    exploration_rate | 0.933    |
| time/               |          |
|    episodes         | 4        |
|    fps              | 6426     |
|    time_elapsed     | 0        |
|    total_timesteps  | 141      |
----------------------------------
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 26.4     |
|    ep_rew_mean      | 26.4     |
|    exploration_rate | 0.9      |
| time/               |          |
|    episodes         | 8        |
|    fps              | 6604     |
|    time_elapsed     | 0        |
|    total_timesteps  | 211      |
----------------------------------
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 28.3     |
|    ep_rew_mean      | 28.3     |
|    exploration_rate | 0.838    |
| time/               | 

----------------------------------
| rollout/            |          |
|    ep_len_mean      | 21.8     |
|    ep_rew_mean      | 21.8     |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 92       |
|    fps              | 7724     |
|    time_elapsed     | 0        |
|    total_timesteps  | 2008     |
----------------------------------
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 21.6     |
|    ep_rew_mean      | 21.6     |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 96       |
|    fps              | 7737     |
|    time_elapsed     | 0        |
|    total_timesteps  | 2073     |
----------------------------------
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 21.5     |
|    ep_rew_mean      | 21.5     |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes       

----------------------------------
| rollout/            |          |
|    ep_len_mean      | 23.8     |
|    ep_rew_mean      | 23.8     |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 180      |
|    fps              | 8457     |
|    time_elapsed     | 0        |
|    total_timesteps  | 4113     |
----------------------------------
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 24.2     |
|    ep_rew_mean      | 24.2     |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 184      |
|    fps              | 8510     |
|    time_elapsed     | 0        |
|    total_timesteps  | 4241     |
----------------------------------
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 24.4     |
|    ep_rew_mean      | 24.4     |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes       

----------------------------------
| rollout/            |          |
|    ep_len_mean      | 23.4     |
|    ep_rew_mean      | 23.4     |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 268      |
|    fps              | 8712     |
|    time_elapsed     | 0        |
|    total_timesteps  | 6123     |
----------------------------------
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 23.2     |
|    ep_rew_mean      | 23.2     |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 272      |
|    fps              | 8708     |
|    time_elapsed     | 0        |
|    total_timesteps  | 6215     |
----------------------------------
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 23.2     |
|    ep_rew_mean      | 23.2     |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes       

----------------------------------
| rollout/            |          |
|    ep_len_mean      | 22.8     |
|    ep_rew_mean      | 22.8     |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 356      |
|    fps              | 8622     |
|    time_elapsed     | 0        |
|    total_timesteps  | 8106     |
----------------------------------
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 22.3     |
|    ep_rew_mean      | 22.3     |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 360      |
|    fps              | 8620     |
|    time_elapsed     | 0        |
|    total_timesteps  | 8181     |
----------------------------------
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 22.5     |
|    ep_rew_mean      | 22.5     |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes       

...

----------------------------------
| rollout/            |          |
|    ep_len_mean      | 22.5     |
|    ep_rew_mean      | 22.5     |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 528      |
|    fps              | 8094     |
|    time_elapsed     | 1        |
|    total_timesteps  | 12076    |
----------------------------------
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 22.7     |
|    ep_rew_mean      | 22.7     |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 532      |
|    fps              | 8099     |
|    time_elapsed     | 1        |
|    total_timesteps  | 12157    |
----------------------------------
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 23       |
|    ep_rew_mean      | 23       |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes       

----------------------------------
| rollout/            |          |
|    ep_len_mean      | 21.3     |
|    ep_rew_mean      | 21.3     |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 616      |
|    fps              | 8149     |
|    time_elapsed     | 1        |
|    total_timesteps  | 13891    |
----------------------------------
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 21       |
|    ep_rew_mean      | 21       |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 620      |
|    fps              | 8152     |
|    time_elapsed     | 1        |
|    total_timesteps  | 13961    |
----------------------------------
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 21.1     |
|    ep_rew_mean      | 21.1     |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes       

----------------------------------
| rollout/            |          |
|    ep_len_mean      | 21       |
|    ep_rew_mean      | 21       |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 704      |
|    fps              | 8209     |
|    time_elapsed     | 1        |
|    total_timesteps  | 15791    |
----------------------------------
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 21.2     |
|    ep_rew_mean      | 21.2     |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 708      |
|    fps              | 8211     |
|    time_elapsed     | 1        |
|    total_timesteps  | 15867    |
----------------------------------
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 21.3     |
|    ep_rew_mean      | 21.3     |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes       

----------------------------------
| rollout/            |          |
|    ep_len_mean      | 21.9     |
|    ep_rew_mean      | 21.9     |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 792      |
|    fps              | 8293     |
|    time_elapsed     | 2        |
|    total_timesteps  | 17752    |
----------------------------------
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 22.1     |
|    ep_rew_mean      | 22.1     |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 796      |
|    fps              | 8300     |
|    time_elapsed     | 2        |
|    total_timesteps  | 17858    |
----------------------------------
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 22.6     |
|    ep_rew_mean      | 22.6     |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes       

----------------------------------
| rollout/            |          |
|    ep_len_mean      | 22.2     |
|    ep_rew_mean      | 22.2     |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 876      |
|    fps              | 8082     |
|    time_elapsed     | 2        |
|    total_timesteps  | 19694    |
----------------------------------
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 22.6     |
|    ep_rew_mean      | 22.6     |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 880      |
|    fps              | 8087     |
|    time_elapsed     | 2        |
|    total_timesteps  | 19786    |
----------------------------------
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 22.8     |
|    ep_rew_mean      | 22.8     |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes       